In [0]:
import pandas as pd

iperf2_3_tcp_alex = pd.read_csv("https://corey.to/capstone/client/Alex/tcp.csv")
iperf2_3_udp_alex = pd.read_csv("https://corey.to/capstone/client/Alex/udp.csv")
iperf2_3_udp3m_alex = pd.read_csv("https://corey.to/capstone/client/Alex/udp3m.csv")

iperf3_multiperf3_alex = pd.read_csv("https://corey.to/capstone/client/Alex/multiperf3.csv")

'''
iperf2_3_tcp_alex.info()
iperf2_3_udp_alex.info()
iperf2_3_udp3m_alex.info()
iperf3_multiperf3_alex.info()
'''

'\niperf2_3_tcp_alex.info()\niperf2_3_udp_alex.info()\niperf2_3_udp3m_alex.info()\niperf3_multiperf3_alex.info()\n'

In [0]:
iperf2_3_tcp_spencer = pd.read_csv("https://corey.to/capstone/client/android-spencer/tcp.csv")
iperf2_3_udp_spencer = pd.read_csv("https://corey.to/capstone/client/android-spencer/udp.csv")
iperf2_3_udp3m_spencer = pd.read_csv("https://corey.to/capstone/client/android-spencer/udp3m.csv")

iperf3_multiperf3_spencer = pd.read_csv("https://corey.to/capstone/client/android-spencer/multiperf3.csv")

'''
iperf2_3_tcp_spencer.info()
iperf2_3_udp_spencer.info()
iperf2_3_udp3m_spencer.info()
iperf3_multiperf3_spencer.info()
'''

'\niperf2_3_tcp_spencer.info()\niperf2_3_udp_spencer.info()\niperf2_3_udp3m_spencer.info()\niperf3_multiperf3_spencer.info()\n'

In [0]:
iperf2_3_udp_corey = pd.read_csv("https://corey.to/capstone/client/mac-corey/udp.csv")
iperf2_3_udp3m_corey = pd.read_csv("https://corey.to/capstone/client/mac-corey/udp3m.csv")
iperf2_3_tcp_corey = pd.read_csv("https://corey.to/capstone/client/mac-corey/tcp.csv")

iperf3_multiperf3_corey = pd.read_csv("https://corey.to/capstone/client/mac-corey/multiperf3.csv")

'''
iperf2_3_udp_corey.info()
iperf2_3_udp3m_corey.info()
iperf2_3_tcp_corey.info()
iperf3_multiperf3_corey.info()
'''

'\niperf2_3_udp_corey.info()\niperf2_3_udp3m_corey.info()\niperf2_3_tcp_corey.info()\niperf3_multiperf3_corey.info()\n'

In [0]:
# imports and functions go here

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import num2date
from datetime import timedelta

def reindex(*dfs, col='unix_timestamp'):
    """
    Reindex dataframes to datetime indexes based on values of `col`.
    Parameters
    ----------
    *dfs : pandas.DataFrame
        Dataframes to be reindexed, one or several. Modified in place.
    col : str, optional
        Column name to use as index. The default is 'unix_timestamp'.
    Returns
    -------
    None.
    """
    for df in dfs:
        if df.index.name is not col:
            df['unix_timestamp'] = pd.to_datetime(df[col], unit='s')
            df.set_index(df['unix_timestamp'], inplace=True)

def merge(*dfs, time=pd.Timedelta('5m')):
    """
    Merge two dataframes by index time within `time` of each other.
    Parameters
    ----------
    *dfs : pandas.DataFrame
        Dataframes to merge. Must have datetime indexes.
    time : pandas.Timedelta, optional
        The time difference range within which to merge rows. The default is pd.Timedelta('5m').
    Returns
    -------
    merged : pandas.DataFrame
        Single dataframe containing colname_x and colname_y values and 'unix_timestamp' column as copy of index.
    """
    merged = dfs[0].copy()
    for df in dfs[1:]:
        merged = pd.merge_asof(merged, df, left_index=True, right_index=True, tolerance=time, direction='nearest')
    merged['unix_timestamp'] = merged.index
    print('merged shape: {}; after dropna: {}'.format(merged.shape, merged.dropna().shape))
    return merged.dropna()

def reindex_and_merge(*dfs, col='unix_timestamp'):
    reindex(*dfs, col=col)
    return merge(*dfs)

def _check_merged(*dfs):
    if len(dfs) == 1:
        return dfs[0]
    else:
        return reindex_and_merge(*dfs)

def get_max_min(df, col='Jitter'):
    max = df[[col+'_x', col+'_y']].max().max()
    min = df[[col+'_x', col+'_y']].min().min()
    return (max, min)

def _pct_diff(i2, i3):
    """
    Calculate percent difference, iPerf 3 vs iPerf 2
    Parameters
    ----------
    i2 : number
        Value for iPerf 2.
    i3 : number
        Value for iPerf 3.
    Returns
    -------
    number
        Percent difference, i3 vs i2
    """
    return ((i3-i2)/i3) * 100

def compare_percentiles(*dfs, col='Jitter', title='Jitter', description='UDP Jitter', percentile=95):
    df = _check_merged(*dfs)
    iperf2_percentile = np.percentile(df[col+'_x'], percentile)
    iperf3_percentile = np.percentile(df[col+'_y'], percentile)

    iperf2_percentile_min = np.percentile(df[col+'_x'], 100-percentile)
    iperf3_percentile_min = np.percentile(df[col+'_y'], 100-percentile)
    pct_diff = _pct_diff(iperf2_percentile, iperf3_percentile)
    
    print(f'iPerf 2 {title}, {percentile}th percentile: {iperf2_percentile:.3f}\n'
          f'iPerf 3 {title}, {percentile}th percentile: {iperf3_percentile:.3f}\n'
          f'Percent difference, iPerf 3 vs iPerf 2 {description} {percentile}th percentile: {pct_diff:.0f}%\n')
    '''
    iperf2_mean = np.std(df[df[col+'_x'] <= iperf2_percentile][col+'_x'])
    iperf3_mean = np.std(df[df[col+'_y'] <= iperf3_percentile][col+'_y'])
    pct_diff = _pct_diff(iperf2_mean, iperf3_mean)
    
    print(f'iPerf 2 {title}, std below {percentile}th percentile: {iperf2_mean:.3f}\n'
          f'iPerf 3 {title}, std below {percentile}th percentile: {iperf3_mean:.3f}\n'
          f'Percent difference, iPerf 3 vs iPerf 2 {description} standard deviation below {percentile}th percentile: {pct_diff:.0f}%\n')
    '''
    iperf2_within_percentile = df[(df[col+'_x'] >= iperf2_percentile_min) & (df[col+'_x'] <= iperf2_percentile)][col+'_x']
    iperf3_within_percentile = df[(df[col+'_y'] >= iperf3_percentile_min) & (df[col+'_y'] <= iperf3_percentile)][col+'_y']

    iperf2_mean_within_percentile = np.mean(iperf2_within_percentile)
    iperf2_std_within_percentile = np.std(iperf2_within_percentile)

    iperf3_mean_within_percentile = np.mean(iperf3_within_percentile)
    iperf3_std_within_percentile = np.std(iperf3_within_percentile)

    mean_pct_diff = _pct_diff(iperf2_mean_within_percentile, iperf3_mean_within_percentile)
    std_pct_diff = _pct_diff(iperf2_std_within_percentile, iperf3_std_within_percentile)

    print(f'iPerf 2, between {100-percentile}-{percentile}th percentiles:\n'
          f'\tMean {description}: {iperf2_mean_within_percentile:.3f}\n'
          f'\tStandard deviation: {iperf2_std_within_percentile:.3f}\n\n'
          f'iPerf 3, between {100-percentile}-{percentile}th percentiles:\n'
          f'\tMean {description}: {iperf3_mean_within_percentile:.3f}\n'
          f'\tStandard deviation: {iperf3_std_within_percentile:.3f}\n\n'
          f'Difference (mean {description}, iPerf 3 vs iPerf 2): {mean_pct_diff:.2f}%\n'
          f'Difference (standard deviation, iPerf 3 vs iPerf 2): {std_pct_diff:.2f}%\n')
    
def get_percentiles(df, col, percentile=95):
    """
    Parameters
    ----------
    df : pandas.DataFrame
        A merged dataframe containing _x and _y columns.
    col : str
        Which column to look at.
    percentile : int, optional
        The percentile to calculate. The default is 95.
    Returns
    -------
    (float64, float64, float64)
        Tuple with percentile of col_x, col_y, and max of the two.
    """
    p1 = np.percentile(df[col + '_x'], percentile)
    p2 = np.percentile(df[col + '_y'], percentile)
    return (p1, p2, np.max([p1, p2]))

def compare_std(*dfs, col='Jitter', title='Jitter', description='UDP Jitter'):
    df = _check_merged(*dfs)
    
    iperf2_std = np.std(df[col+'_x'])
    iperf3_std = np.std(df[col+'_y'])
    
    pct_diff = ((iperf3_std - iperf2_std)/iperf3_std) * 100
    
    print(f'iPerf 2 {title}, standard deviation: {iperf2_std:.3f}\n'
          f'iPerf 3 {title}, standard deviation: {iperf3_std:.3f}\n'
          f'Percent difference, iPerf 3 vs iPerf 2 {description} standard deviation: {pct_diff:.0f}%')


Here we do the data analysis (for graphs, we use the web interface from the web/ directory on the GitHub).

UDP analysis first.

In [69]:
# Spencer/Android 8 (UDP)
print(iperf2_3_udp_spencer.shape)
compare_std(iperf2_3_udp_spencer, col='jitter_ms')
compare_percentiles(iperf2_3_udp_spencer, col='jitter_ms')
compare_percentiles(iperf2_3_udp_spencer, col='jitter_ms', percentile=99)

(344, 14)
iPerf 2 Jitter, standard deviation: 6.779
iPerf 3 Jitter, standard deviation: 1.568
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter standard deviation: -332%
iPerf 2 Jitter, 95th percentile: 4.792
iPerf 3 Jitter, 95th percentile: 4.617
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter 95th percentile: -4%

iPerf 2, between 5-95th percentiles:
	Mean UDP Jitter: 1.906
	Standard deviation: 0.978

iPerf 3, between 5-95th percentiles:
	Mean UDP Jitter: 1.823
	Standard deviation: 0.883

Difference (mean UDP Jitter, iPerf 3 vs iPerf 2): -4.57%
Difference (standard deviation, iPerf 3 vs iPerf 2): -10.70%

iPerf 2 Jitter, 99th percentile: 42.139
iPerf 3 Jitter, 99th percentile: 8.895
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter 99th percentile: -374%

iPerf 2, between 1-99th percentiles:
	Mean UDP Jitter: 2.047
	Standard deviation: 1.425

iPerf 3, between 1-99th percentiles:
	Mean UDP Jitter: 1.956
	Standard deviation: 1.259

Difference (mean UDP Jitter, iPerf 3 vs iPerf 2): -4

In [70]:
# Spencer (UDP 3m)
print(iperf2_3_udp3m_spencer.shape)
compare_std(iperf2_3_udp3m_spencer, col='jitter_ms')
compare_percentiles(iperf2_3_udp3m_spencer, col='jitter_ms')
compare_percentiles(iperf2_3_udp3m_spencer, col='jitter_ms', percentile=99)

(342, 14)
iPerf 2 Jitter, standard deviation: 6.794
iPerf 3 Jitter, standard deviation: 0.614
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter standard deviation: -1006%
iPerf 2 Jitter, 95th percentile: 2.757
iPerf 3 Jitter, 95th percentile: 2.556
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter 95th percentile: -8%

iPerf 2, between 5-95th percentiles:
	Mean UDP Jitter: 1.321
	Standard deviation: 0.458

iPerf 3, between 5-95th percentiles:
	Mean UDP Jitter: 1.357
	Standard deviation: 0.395

Difference (mean UDP Jitter, iPerf 3 vs iPerf 2): 2.68%
Difference (standard deviation, iPerf 3 vs iPerf 2): -15.81%

iPerf 2 Jitter, 99th percentile: 44.786
iPerf 3 Jitter, 99th percentile: 3.339
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter 99th percentile: -1241%

iPerf 2, between 1-99th percentiles:
	Mean UDP Jitter: 1.418
	Standard deviation: 1.138

iPerf 3, between 1-99th percentiles:
	Mean UDP Jitter: 1.385
	Standard deviation: 0.511

Difference (mean UDP Jitter, iPerf 3 vs iPerf 2): -

In [71]:
# Alex/Windows (UDP)
print(iperf2_3_udp_alex.shape)
compare_std(iperf2_3_udp_alex, col='jitter_ms')
compare_percentiles(iperf2_3_udp_alex, col='jitter_ms')
compare_percentiles(iperf2_3_udp_alex, col='jitter_ms', percentile=99)

(366, 14)
iPerf 2 Jitter, standard deviation: 8.100
iPerf 3 Jitter, standard deviation: 0.529
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter standard deviation: -1431%
iPerf 2 Jitter, 95th percentile: 1.434
iPerf 3 Jitter, 95th percentile: 1.478
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter 95th percentile: 3%

iPerf 2, between 5-95th percentiles:
	Mean UDP Jitter: 1.012
	Standard deviation: 0.167

iPerf 3, between 5-95th percentiles:
	Mean UDP Jitter: 1.022
	Standard deviation: 0.174

Difference (mean UDP Jitter, iPerf 3 vs iPerf 2): 1.02%
Difference (standard deviation, iPerf 3 vs iPerf 2): 4.05%

iPerf 2 Jitter, 99th percentile: 33.060
iPerf 3 Jitter, 99th percentile: 3.743
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter 99th percentile: -783%

iPerf 2, between 1-99th percentiles:
	Mean UDP Jitter: 1.131
	Standard deviation: 1.241

iPerf 3, between 1-99th percentiles:
	Mean UDP Jitter: 1.079
	Standard deviation: 0.425

Difference (mean UDP Jitter, iPerf 3 vs iPerf 2): -4.82

In [72]:
# Alex (UDP 3m)
print(iperf2_3_udp3m_alex.shape)
compare_std(iperf2_3_udp3m_alex, col='jitter_ms')
compare_percentiles(iperf2_3_udp3m_alex, col='jitter_ms')
compare_percentiles(iperf2_3_udp3m_alex, col='jitter_ms', percentile=99)

(368, 14)
iPerf 2 Jitter, standard deviation: 7.976
iPerf 3 Jitter, standard deviation: 0.607
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter standard deviation: -1214%
iPerf 2 Jitter, 95th percentile: 1.013
iPerf 3 Jitter, 95th percentile: 0.890
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter 95th percentile: -14%

iPerf 2, between 5-95th percentiles:
	Mean UDP Jitter: 0.580
	Standard deviation: 0.133

iPerf 3, between 5-95th percentiles:
	Mean UDP Jitter: 0.588
	Standard deviation: 0.125

Difference (mean UDP Jitter, iPerf 3 vs iPerf 2): 1.42%
Difference (standard deviation, iPerf 3 vs iPerf 2): -6.37%

iPerf 2 Jitter, 99th percentile: 62.923
iPerf 3 Jitter, 99th percentile: 1.761
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter 99th percentile: -3472%

iPerf 2, between 1-99th percentiles:
	Mean UDP Jitter: 0.997
	Standard deviation: 4.753

iPerf 3, between 1-99th percentiles:
	Mean UDP Jitter: 0.602
	Standard deviation: 0.177

Difference (mean UDP Jitter, iPerf 3 vs iPerf 2): -

In [73]:
# Corey/macOS (UDP)
print(iperf2_3_udp_corey.shape)
compare_std(iperf2_3_udp_corey, col='jitter_ms')
compare_percentiles(iperf2_3_udp_corey, col='jitter_ms')
compare_percentiles(iperf2_3_udp_corey, col='jitter_ms', percentile=99)

(366, 14)
iPerf 2 Jitter, standard deviation: 11.336
iPerf 3 Jitter, standard deviation: 3.356
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter standard deviation: -238%
iPerf 2 Jitter, 95th percentile: 11.116
iPerf 3 Jitter, 95th percentile: 11.094
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter 95th percentile: -0%

iPerf 2, between 5-95th percentiles:
	Mean UDP Jitter: 3.552
	Standard deviation: 1.899

iPerf 3, between 5-95th percentiles:
	Mean UDP Jitter: 3.925
	Standard deviation: 2.061

Difference (mean UDP Jitter, iPerf 3 vs iPerf 2): 9.51%
Difference (standard deviation, iPerf 3 vs iPerf 2): 7.87%

iPerf 2 Jitter, 99th percentile: 65.979
iPerf 3 Jitter, 99th percentile: 14.884
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter 99th percentile: -343%

iPerf 2, between 1-99th percentiles:
	Mean UDP Jitter: 5.064
	Standard deviation: 9.380

iPerf 3, between 1-99th percentiles:
	Mean UDP Jitter: 4.148
	Standard deviation: 2.668

Difference (mean UDP Jitter, iPerf 3 vs iPerf 2): -

In [74]:
# Corey (UDP 3m)
print(iperf2_3_udp3m_corey.shape)
compare_std(iperf2_3_udp3m_corey, col='jitter_ms')
compare_percentiles(iperf2_3_udp3m_corey, col='jitter_ms')
compare_percentiles(iperf2_3_udp3m_corey, col='jitter_ms', percentile=99)

(363, 14)
iPerf 2 Jitter, standard deviation: 9.261
iPerf 3 Jitter, standard deviation: 1.620
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter standard deviation: -472%
iPerf 2 Jitter, 95th percentile: 6.049
iPerf 3 Jitter, 95th percentile: 5.918
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter 95th percentile: -2%

iPerf 2, between 5-95th percentiles:
	Mean UDP Jitter: 2.652
	Standard deviation: 1.066

iPerf 3, between 5-95th percentiles:
	Mean UDP Jitter: 2.654
	Standard deviation: 0.919

Difference (mean UDP Jitter, iPerf 3 vs iPerf 2): 0.07%
Difference (standard deviation, iPerf 3 vs iPerf 2): -16.01%

iPerf 2 Jitter, 99th percentile: 65.121
iPerf 3 Jitter, 99th percentile: 9.639
Percent difference, iPerf 3 vs iPerf 2 UDP Jitter 99th percentile: -576%

iPerf 2, between 1-99th percentiles:
	Mean UDP Jitter: 3.427
	Standard deviation: 6.623

iPerf 3, between 1-99th percentiles:
	Mean UDP Jitter: 2.794
	Standard deviation: 1.379

Difference (mean UDP Jitter, iPerf 3 vs iPerf 2): -22

TCP analysis follows.

In [75]:
# Spencer/Android 8 (TCP)
print(iperf2_3_tcp_spencer.shape)
compare_std(iperf2_3_tcp_spencer, col='transfer_rate', title='Transfer Rate', description='TCP throughput')
compare_percentiles(iperf2_3_tcp_spencer, col='transfer_rate', title='Transfer Rate', description='TCP throughput')
compare_percentiles(iperf2_3_tcp_spencer, col='transfer_rate', title='Transfer Rate', description='TCP throughput', percentile=99)

(342, 13)
iPerf 2 Transfer Rate, standard deviation: 1175926.034
iPerf 3 Transfer Rate, standard deviation: 1152513.411
Percent difference, iPerf 3 vs iPerf 2 TCP throughput standard deviation: -2%
iPerf 2 Transfer Rate, 95th percentile: 10723747.350
iPerf 3 Transfer Rate, 95th percentile: 10437803.814
Percent difference, iPerf 3 vs iPerf 2 TCP throughput 95th percentile: -3%

iPerf 2, between 5-95th percentiles:
	Mean TCP throughput: 9918493.706
	Standard deviation: 903671.555

iPerf 3, between 5-95th percentiles:
	Mean TCP throughput: 9766018.126
	Standard deviation: 840561.985

Difference (mean TCP throughput, iPerf 3 vs iPerf 2): -1.56%
Difference (standard deviation, iPerf 3 vs iPerf 2): -7.51%

iPerf 2 Transfer Rate, 99th percentile: 10749027.640
iPerf 3 Transfer Rate, 99th percentile: 10442114.309
Percent difference, iPerf 3 vs iPerf 2 TCP throughput 99th percentile: -3%

iPerf 2, between 1-99th percentiles:
	Mean TCP throughput: 9828761.847
	Standard deviation: 1067699.453

iPe

In [76]:
# Alex/Windows (TCP)
print(iperf2_3_tcp_alex.shape)
compare_std(iperf2_3_tcp_alex, col='transfer_rate', title='Transfer Rate', description='TCP throughput')
compare_percentiles(iperf2_3_tcp_alex, col='transfer_rate', title='Transfer Rate', description='TCP throughput')
compare_percentiles(iperf2_3_tcp_alex, col='transfer_rate', title='Transfer Rate', description='TCP throughput', percentile=99)

(358, 13)
iPerf 2 Transfer Rate, standard deviation: 217163.230
iPerf 3 Transfer Rate, standard deviation: 335407.758
Percent difference, iPerf 3 vs iPerf 2 TCP throughput standard deviation: 35%
iPerf 2 Transfer Rate, 95th percentile: 4926228.550
iPerf 3 Transfer Rate, 95th percentile: 4680158.407
Percent difference, iPerf 3 vs iPerf 2 TCP throughput 95th percentile: -5%

iPerf 2, between 5-95th percentiles:
	Mean TCP throughput: 4877424.348
	Standard deviation: 58520.278

iPerf 3, between 5-95th percentiles:
	Mean TCP throughput: 4636329.427
	Standard deviation: 52255.418

Difference (mean TCP throughput, iPerf 3 vs iPerf 2): -5.20%
Difference (standard deviation, iPerf 3 vs iPerf 2): -11.99%

iPerf 2 Transfer Rate, 99th percentile: 4927649.320
iPerf 3 Transfer Rate, 99th percentile: 4712772.297
Percent difference, iPerf 3 vs iPerf 2 TCP throughput 99th percentile: -5%

iPerf 2, between 1-99th percentiles:
	Mean TCP throughput: 4867543.226
	Standard deviation: 89011.259

iPerf 3, bet

In [77]:
# Corey/macOS (TCP)
print(iperf2_3_tcp_corey.shape)
compare_std(iperf2_3_tcp_corey, col='transfer_rate', title='Transfer Rate', description='TCP throughput')
compare_percentiles(iperf2_3_tcp_corey, col='transfer_rate', title='Transfer Rate', description='TCP throughput')
compare_percentiles(iperf2_3_tcp_corey, col='transfer_rate', title='Transfer Rate', description='TCP throughput', percentile=99)

(365, 13)
iPerf 2 Transfer Rate, standard deviation: 5952700.183
iPerf 3 Transfer Rate, standard deviation: 5879835.276
Percent difference, iPerf 3 vs iPerf 2 TCP throughput standard deviation: -1%
iPerf 2 Transfer Rate, 95th percentile: 17928031.800
iPerf 3 Transfer Rate, 95th percentile: 17644731.491
Percent difference, iPerf 3 vs iPerf 2 TCP throughput 95th percentile: -2%

iPerf 2, between 5-95th percentiles:
	Mean TCP throughput: 10766876.428
	Standard deviation: 5590054.750

iPerf 3, between 5-95th percentiles:
	Mean TCP throughput: 10415767.210
	Standard deviation: 5532021.390

Difference (mean TCP throughput, iPerf 3 vs iPerf 2): -3.37%
Difference (standard deviation, iPerf 3 vs iPerf 2): -1.05%

iPerf 2 Transfer Rate, 99th percentile: 18002166.120
iPerf 3 Transfer Rate, 99th percentile: 17719039.318
Percent difference, iPerf 3 vs iPerf 2 TCP throughput 99th percentile: -2%

iPerf 2, between 1-99th percentiles:
	Mean TCP throughput: 10674657.409
	Standard deviation: 5872753.784

Extra: iperf3 vs multiperf3 analysis.

\*\*Here, "iPerf 2" means "iPerf 3", and "iPerf 3" means "multiperf3"

In [78]:
# Spencer
print(iperf3_multiperf3_spencer.shape)
compare_std(iperf3_multiperf3_spencer, col='transfer_rate', title='Transfer Rate', description='TCP throughput')
compare_percentiles(iperf3_multiperf3_spencer, col='transfer_rate', title='Transfer Rate', description='TCP throughput')
compare_percentiles(iperf3_multiperf3_spencer, col='transfer_rate', title='Transfer Rate', description='TCP throughput', percentile=99)

(341, 10)
iPerf 2 Transfer Rate, standard deviation: 1154006.348
iPerf 3 Transfer Rate, standard deviation: 1179497.153
Percent difference, iPerf 3 vs iPerf 2 TCP throughput standard deviation: 2%
iPerf 2 Transfer Rate, 95th percentile: 10437817.368
iPerf 3 Transfer Rate, 95th percentile: 10473679.296
Percent difference, iPerf 3 vs iPerf 2 TCP throughput 95th percentile: 0%

iPerf 2, between 5-95th percentiles:
	Mean TCP throughput: 9759245.972
	Standard deviation: 852001.303

iPerf 3, between 5-95th percentiles:
	Mean TCP throughput: 9780762.899
	Standard deviation: 837517.352

Difference (mean TCP throughput, iPerf 3 vs iPerf 2): 0.22%
Difference (standard deviation, iPerf 3 vs iPerf 2): -1.73%

iPerf 2 Transfer Rate, 99th percentile: 10442130.684
iPerf 3 Transfer Rate, 99th percentile: 10483439.647
Percent difference, iPerf 3 vs iPerf 2 TCP throughput 99th percentile: 0%

iPerf 2, between 1-99th percentiles:
	Mean TCP throughput: 9670573.807
	Standard deviation: 1012400.061

iPerf 3

In [79]:
# Alex
print(iperf3_multiperf3_alex.shape)
compare_std(iperf3_multiperf3_alex, col='transfer_rate', title='Transfer Rate', description='TCP throughput')
compare_percentiles(iperf3_multiperf3_alex, col='transfer_rate', title='Transfer Rate', description='TCP throughput')
compare_percentiles(iperf3_multiperf3_alex, col='transfer_rate', title='Transfer Rate', description='TCP throughput', percentile=99)

(355, 10)
iPerf 2 Transfer Rate, standard deviation: 73715.087
iPerf 3 Transfer Rate, standard deviation: 124321.930
Percent difference, iPerf 3 vs iPerf 2 TCP throughput standard deviation: 41%
iPerf 2 Transfer Rate, 95th percentile: 4680162.766
iPerf 3 Transfer Rate, 95th percentile: 4718704.559
Percent difference, iPerf 3 vs iPerf 2 TCP throughput 95th percentile: 1%

iPerf 2, between 5-95th percentiles:
	Mean TCP throughput: 4637464.932
	Standard deviation: 50759.917

iPerf 3, between 5-95th percentiles:
	Mean TCP throughput: 4675907.264
	Standard deviation: 50664.259

Difference (mean TCP throughput, iPerf 3 vs iPerf 2): 0.82%
Difference (standard deviation, iPerf 3 vs iPerf 2): -0.19%

iPerf 2 Transfer Rate, 99th percentile: 4712804.367
iPerf 3 Transfer Rate, 99th percentile: 4720092.250
Percent difference, iPerf 3 vs iPerf 2 TCP throughput 99th percentile: 0%

iPerf 2, between 1-99th percentiles:
	Mean TCP throughput: 4631681.331
	Standard deviation: 64179.886

iPerf 3, between 

In [80]:
# Corey
print(iperf3_multiperf3_corey.shape)
compare_std(iperf3_multiperf3_corey, col='transfer_rate', title='Transfer Rate', description='TCP throughput')
compare_percentiles(iperf3_multiperf3_corey, col='transfer_rate', title='Transfer Rate', description='TCP throughput')
compare_percentiles(iperf3_multiperf3_corey, col='transfer_rate', title='Transfer Rate', description='TCP throughput', percentile=99)

(364, 10)
iPerf 2 Transfer Rate, standard deviation: 5867534.626
iPerf 3 Transfer Rate, standard deviation: 5889771.677
Percent difference, iPerf 3 vs iPerf 2 TCP throughput standard deviation: 0%
iPerf 2 Transfer Rate, 95th percentile: 17644751.414
iPerf 3 Transfer Rate, 95th percentile: 17656400.929
Percent difference, iPerf 3 vs iPerf 2 TCP throughput 95th percentile: 0%

iPerf 2, between 5-95th percentiles:
	Mean TCP throughput: 10441782.374
	Standard deviation: 5520491.441

iPerf 3, between 5-95th percentiles:
	Mean TCP throughput: 10518382.853
	Standard deviation: 5542356.664

Difference (mean TCP throughput, iPerf 3 vs iPerf 2): 0.73%
Difference (standard deviation, iPerf 3 vs iPerf 2): 0.39%

iPerf 2 Transfer Rate, 99th percentile: 17719054.153
iPerf 3 Transfer Rate, 99th percentile: 17746316.159
Percent difference, iPerf 3 vs iPerf 2 TCP throughput 99th percentile: 0%

iPerf 2, between 1-99th percentiles:
	Mean TCP throughput: 10367495.069
	Standard deviation: 5789475.452

iPe